In [1]:
from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoModelForMaskedLM, AutoTokenizer

config = AutoConfig.from_pretrained('vinai/bertweet-base')
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')
model = AutoModelForMaskedLM.from_config(config)

Downloading: 100%|██████████| 558/558 [00:00<00:00, 279kB/s]
Downloading: 100%|██████████| 824k/824k [00:05<00:00, 165kB/s]  
Downloading: 100%|██████████| 1.03M/1.03M [00:09<00:00, 112kB/s] 
emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import pandas as pd
from glob import glob

def data_loading(url):
    with open(url, 'r', encoding='utf-8') as f:
        # data = json.loads(f.read())
        df = pd.read_json(f)
        data = df.copy()
        # data = df.loc[:, ['sentiment', 'body']]
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['label'] = data['sentiment'].cat.codes
        data = data.rename(columns={'sentiment': 'labels', 'body': 'sentense'})

        symbols = set()
        for symbol_list in data['sentense'].str.findall(r'\$[A-Z]+'):
            for symbol in symbol_list:
                symbols.add(symbol)
        return data, symbols


data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = data_loading(url)
special_tokens_dict = {'additional_special_tokens': list(symbols)}
tokenizer.add_special_tokens(special_tokens_dict)

100

In [33]:
data.loc[data['sentense'].str.find('http') != -1].head()

,message_id,sentense,labels,created_at,label
17,447646810,$ETH.X $AAPL $CAPR \n \n$1800 into $54k in th...,Bullish,2022-03-28 13:41:04+00:00,1
37,447639081,$NVDA Small and big investors! this boat about...,Bullish,2022-03-28 13:21:18+00:00,1
39,447636617,$CODX $AGRX $SHOP $AMD $NVDA https://finance.y...,Bullish,2022-03-28 13:13:13+00:00,1
45,447633466,$AMD - $NVDA \n\nhttps://stocktwits.com/jantho...,Bullish,2022-03-28 13:02:56+00:00,1
70,447633241,$PLTR $MSFT $AMZN $GOOGL $QQQ \n\nhttps://clou...,Bullish,2022-03-28 13:02:13+00:00,1


In [47]:
import re
from parse import normalizeTweet
sentense = data.loc[37]['sentense']

s = re.split(r'(\$[A-Z]+)', sentense)
a = [tokenizer.normalizeTweet(i) if i.find('$') == -1 else i for i in s]
new_s = " ".join(a)

print(tokenizer.normalizeTweet('<mask> to the sky !!!'))

# print(sentense)
# print(tokenizer.encode(sentense))
# print(tokenizer.normalizeTweet(sentense))
# print(tokenizer(sentense))
# print(normalizeTweet(sentense, tokenizer))
# print(tokenizer.encode(tokenizer.normalizeTweet(sentense)))

<mask> to the sky ! ! !


In [44]:
tokenizer.convert_ids_to_tokens(55350)

'AMZN'

In [6]:
import wandb
api = wandb.Api()
run = api.run("Stocktwit_sentiment_analysis/")
# run.delete()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29625, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [2]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element